In [4]:
# learning async and await concept 
import nest_asyncio
import asyncio
import time
nest_asyncio.apply()

async def simulate_task(name, delay):
    print(f"Task {name}: started, will sleep for {delay} seconds")
    await asyncio.sleep(delay)
    print(f"Task {name}: finished after {delay} seconds")

async def main():
    t1=time.time()
    tasks = [asyncio.create_task(simulate_task(f"Task_{i+1}", i+1)) for i in range(10)]
    print("Main: tasks scheduled, awaiting their completion")
    await asyncio.gather(*tasks)
    t2 = time.time()
    print(f"Main: all tasks completed in {t2 - t1:.2f} seconds")

# Now we can safely call asyncio.run
asyncio.run(main())

Main: tasks scheduled, awaiting their completion
Task Task_1: started, will sleep for 1 seconds
Task Task_2: started, will sleep for 2 seconds
Task Task_3: started, will sleep for 3 seconds
Task Task_4: started, will sleep for 4 seconds
Task Task_5: started, will sleep for 5 seconds
Task Task_6: started, will sleep for 6 seconds
Task Task_7: started, will sleep for 7 seconds
Task Task_8: started, will sleep for 8 seconds
Task Task_9: started, will sleep for 9 seconds
Task Task_10: started, will sleep for 10 seconds
Task Task_1: finished after 1 seconds
Task Task_2: finished after 2 seconds
Task Task_3: finished after 3 seconds
Task Task_4: finished after 4 seconds
Task Task_5: finished after 5 seconds
Task Task_6: finished after 6 seconds
Task Task_7: finished after 7 seconds
Task Task_8: finished after 8 seconds
Task Task_9: finished after 9 seconds
Task Task_10: finished after 10 seconds
Main: all tasks completed in 10.00 seconds


In [5]:
import time

def simulate_task(name, delay):
    print(f"Task {name}: started, will sleep for {delay} seconds")
    time.sleep(delay)
    print(f"Task {name}: finished after {delay} seconds")

def main():
    t1 = time.time()
    print("Main: starting tasks sequentially")
    for i in range(10):
        simulate_task(f"Task_{i+1}", i+1)
    t2 = time.time()
    print(f"Main: all tasks completed in {t2 - t1:.2f} seconds")
    print("Main: all tasks completed")

if __name__ == "__main__":
    main()

Main: starting tasks sequentially
Task Task_1: started, will sleep for 1 seconds
Task Task_1: finished after 1 seconds
Task Task_2: started, will sleep for 2 seconds
Task Task_2: finished after 2 seconds
Task Task_3: started, will sleep for 3 seconds
Task Task_3: finished after 3 seconds
Task Task_4: started, will sleep for 4 seconds
Task Task_4: finished after 4 seconds
Task Task_5: started, will sleep for 5 seconds
Task Task_5: finished after 5 seconds
Task Task_6: started, will sleep for 6 seconds
Task Task_6: finished after 6 seconds
Task Task_7: started, will sleep for 7 seconds
Task Task_7: finished after 7 seconds
Task Task_8: started, will sleep for 8 seconds
Task Task_8: finished after 8 seconds
Task Task_9: started, will sleep for 9 seconds
Task Task_9: finished after 9 seconds
Task Task_10: started, will sleep for 10 seconds
Task Task_10: finished after 10 seconds
Main: all tasks completed in 55.01 seconds
Main: all tasks completed


In [11]:
import requests

# 1. Open the SSE stream
url = "http://localhost:8001/sse"
headers = {
    "Accept": "text/event-stream",   # tell the server we want an SSE stream
}

# stream=True keeps the connection open so we can iterate over incoming events
with requests.get(url, headers=headers, stream=True) as resp:
    resp.raise_for_status()

    # 2. Iterate over each line sent by the server
    for raw_line in resp.iter_lines():
        print("→ raw SSE event:", raw_line)
        # if not raw_line:
        #     # blank line = end of one SSE “message”
        #     continue
        # line = raw_line.decode("utf-8")
        # if line.startswith("data:"):
        #     # strip off the "data: " prefix
        #     payload = line[len("data: "):]
        #     print("← SSE event payload:", payload)

→ raw SSE event: b'event: endpoint'
→ raw SSE event: b'data: /messages/?session_id=1f87d37c282b453aacccf01e0392234a'
→ raw SSE event: b''
→ raw SSE event: b': ping - 2025-06-15 16:44:18.374139+00:00'
→ raw SSE event: b''
→ raw SSE event: b': ping - 2025-06-15 16:44:33.375552+00:00'
→ raw SSE event: b''
→ raw SSE event: b'event: message'
→ raw SSE event: b'data: {"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":false}},"serverInfo":{"name":"wiki","version":"1.5.0"}}}'
→ raw SSE event: b''
→ raw SSE event: b': ping - 2025-06-15 16:44:48.376331+00:00'
→ raw SSE event: b''


In [9]:
from sseclient import SSEClient
import json

EVENTS = SSEClient(
    "http://localhost:8001/sse",
    headers={"Accept": "text/event-stream"},
)

for evt in EVENTS:
    if evt.event == "endpoint":
        post_path = evt.data  # e.g. "/messages/?session_id=…"
        break


ModuleNotFoundError: No module named 'sseclient'

In [1]:
import requests, json
from sseclient import SSEClient

BASE = "http://localhost:8001"

# 1) Start SSE & grab session_id
events = SSEClient(f"{BASE}/sse", headers={"Accept":"text/event-stream"})
for evt in events:
    if evt.event == "endpoint":
        post_path = evt.data               # e.g. "/messages/?session_id=XYZ"
        break

# Helper to post and (optionally) ignore empty body
def post_rpc(payload):
    return requests.post(f"{BASE}{post_path}", json=payload)

# 2) Initialize
init = {
  "jsonrpc": "2.0",
  "method":  "initialize",
  "params":  {
    "protocolVersion": "1.0",
    "clientInfo":      {"name":"cli","version":"0.1"},
    "capabilities":    {}
  },
  "id": 1
}
post_rpc(init)

# 3) Wait for initialize response (id=1)
for evt in events:
    print("→ raw SSE event:", evt.data)
    if evt.data and json.loads(evt.data).get("id") == 1:
        print("Initialized:", evt.data)
        break

# 4) Now you can list tools
# tools_list = {
#   "jsonrpc": "2.0",
#   "method":  "tools/list",
#   "params":  {},
#   "id":      2
# }
# post_rpc(tools_list)

# # 5) Read tools/list response (id=2)
# for evt in events:
#     if evt.data:
#         print("→ raw SSE event:", evt.data)
#         # msg = json.loads(evt.data)
#         # if msg.get("id") == 2:
#         #     print("Tools:", msg["result"])
#         #     break




→ raw SSE event: {"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":false}},"serverInfo":{"name":"wiki","version":"1.5.0"}}}
Initialized: {"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":false}},"serverInfo":{"name":"wiki","version":"1.5.0"}}}


In [2]:
# 6) Finally call your sum tool
call = {
  "jsonrpc": "2.0",
  "method":  "tools/call",
  "params":  {"name":"sum","args":{"x":42,"y":58},"kwargs":{}},
  "id":      3
}
post_rpc(call)
print("end= call")

# 7) Read sum result (id=3)
for evt in events:
    if evt.data:
        print("→ raw SSE event:", evt.data)
        # msg = json.loads(evt.data)
        # if msg.get("id") == 3:
        #     print("Sum result:", msg["result"])
        #     break


#events.close()

end= call

→ raw SSE event: /messages/?session_id=98570b4137b049d6bfd8b71e95bd2405


KeyboardInterrupt: 

In [1]:
import requests, json
from sseclient import SSEClient

BASE = "http://localhost:8001"

# 1) Open SSE & grab session_id
sse = SSEClient(f"{BASE}/sse", headers={"Accept":"text/event-stream"})
session_path = None
for evt in sse:
    if evt.event == "endpoint":
        session_path = evt.data             # e.g. "/messages/?session_id=…"
        break

def post_rpc(payload):
    return requests.post(f"{BASE}{session_path}", json=payload)

# 2) Send initialize (id=1)
init = {
    "jsonrpc": "2.0",
    "method":  "initialize",
    "params":  {
        "protocolVersion": "1.0",
        "clientInfo":      {"name":"cli","version":"0.1"},
        "capabilities":    {}
    },
    "id": 1
}
post_rpc(init)

# 3) Prepare your call_sum (use id=2)
call_sum = {
    "jsonrpc": "2.0",
    "method":  "tools/call",
    "params":  {
        "name":   "sum",
        "args":   {"x": 42, "y": 58},
        "kwargs": {}
    },
    "id":      2
}

# 4) Drive the loop: when you see initialize's response, fire call_sum
for evt in sse:
    if not evt.data and "message" not in evt.event:
        continue
    msg = json.loads(evt.data)
    if msg.get("id") == 1:
        # initialization complete → now call sum
        post_rpc(call_sum)
        continue

    if msg.get("id") == 2:
        # sum result arrived
        print("➕ sum(42,58) =", msg["result"])
        break

sse.close()


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import json
from sseclient import SSEClient

sse = SSEClient("http://localhost:8001/sse", headers={"Accept":"text/event-stream"})

# … after grabbing session_path and posting your RPC …

for evt in sse:
    raw = (evt.data or "").strip()
    # skip blank or non-JSON events
    if not raw or not raw.startswith("{"):
        continue

    try:
        msg = json.loads(raw)
    except json.JSONDecodeError:
        # not valid JSON, ignore
        continue

    if msg.get("id") == 1:
        # initialization complete → now call sum
        post_rpc(call_sum)
        continue

    if msg.get("id") == 2:
        # sum result arrived
        print("➕ sum(42,58) =", msg["result"])
        break

sse.close()

SyntaxError: invalid character '…' (U+2026) (472672297.py, line 23)